- Utterance-level representation

In [ ]:
language = "Hindi"
text = f"""I would like you to output a similar pronunciation of the given English using the target language.
For example, assuming the English text is "Author of the danger trail Philip Steels etc".
The phoneme sequence is "ˈɔːθɚɹ ʌvðə dˈeɪndʒɚ tɹˈeɪl fˈɪlɪp stˈiːlz ɛtsˈɛtɹə".
The expected Japanese output is "オーサー・オブ・ザ・デンジャー・トレイル・フィリップ・スティル・エトセトラ。".
The expected Hindi output is "औथर ऑफ द डेंजर ट्रेल फिलिप स्टिल एट सेटरा".

Now, I would like you to find a similar pronunciation of "I am playing a single hand in what looks like a losing game." using {language} characters.
The phoneme sequence is "aɪɐm plˈeɪɪŋ ɐ sˈɪŋɡəl hˈænd ɪn wˌʌt lˈʊks lˈaɪk ɐ lˈuːzɪŋ ɡˈeɪm".
Please represent the phoneme sequence using {language} characters.

All the responses should be in a JSON format as follows:
{{
"English text":[English text],
"phoneme sequence":[phoneme sequence],
"{language} characters":[phoneme sequence represented by {language} characters],
}}"""
print(text)

In [ ]:
language = "English"
text = f"""I would like you to output a similar pronunciation of the given Japanese using the target language (transliteration).
For example, assuming the Japanese text is "Author of the danger trail Philip Steels etc".

Now, I would like you to find a similar pronunciation of "こんにちは、私の名前はツミキです。" using {language} characters.
Please represent the text using {language} characters.

All the responses should be in a JSON format as follows:
{{
"Japanese text":[Japanese text],
"{language} characters":[Japanese text represented by {language} characters],
}}"""
print(text)

- [target language] phoneme sequence -> English sentence 

In [ ]:
sequence = "ˈɔːtʰəɾ ˈɔpʰ dˈə ɖˈẽːɟəɾ ʈɾˈeːl pʰˈɪlɪp sʈˈɪl ˈeːʈ sˈeːʈɾˌaː dˈʌɳɖ"
text = f"""Your task is to generate the corresponding English sentence from the phoneme sequence in IPA format.
For example, assuming the phoneme sequence is "aɪɐm plˈeɪɪŋ ɐ sˈɪŋɡəl hˈænd ɪn wˌʌt lˈʊks lˈaɪk ɐ lˈuːzɪŋ ɡˈeɪm"
Then, the expected English text is "I am playing a single hand in what looks like a losing game".
Here, you task is to generate the English sentence of the phoneme sequence of Hindi sentence, delimited by triple backticks.

```
{sequence}
```"""
print(text)

- generate several word-level representations.

In [ ]:
from phonemizer import phonemize
import random
import pandas as pd
import collections
import numpy as np

In [ ]:
The following is the example in Japanese.
{{
  "I": {{
    "phonemes": "ˈaɪ",
    "choices": ["アィ", "アイ", "アーイ"],
    "similarity order": ["アイ", "アィ", ""],
    }},
  "love": {{
    "phonemes": "lˈʌv",
    "choices": ["प्यार", "इश्क़", "मोहब्बत"],
    "similarity order": ["प्यार", "मोहब्बत", "इश्क़"],
    }},
  "you": {{
    "phonemes": "juː",
    "choices": ["तुझको", "तुझे", "तुम"],
    "similarity order": ["तुम", "तुझे", "तुझको"],
    }},
}}

In [ ]:
sentence = "Author of the danger trail philip steels et cetera"
# sentence = "I am playing a single hand in what looks like a losing game"
# sentence = "I love you"
words = sentence.split()
# phonemized = [phonemize(word, language='en-us', backend='espeak', with_stress=True).split()[0] for word in words]
shfflephonemized = phonemized
# random.shuffle(shfflephonemized)
# language = "Russian"
# language = "Hindi"
language = "Swedish"

start = f"""Can you provide me with three {language} words to represent the phoneme sequences delimited by triple backticks. 
For example, in Japanese, "Trail (tɹˈeɪl)" is expected to have Japanese representation of "トレイル"; where "'" in phonemes represents the stress point of the word. 
Here, your task is to provide me with three {language} words that can replace the phoneme senquences, delimited by triple backticks.
Please focus on phonetically similar characters instead of similar characters in terms of the meaning.
The expected output should be in JSON format. 
You can first list three possible choices of the words and then re-order them in order of the similarity of the pronunciation. 
The following is the example in Hindi language.
{{
  "I": {{
    "phonemes": "ˈaɪ",
    "choices": ["आई", "ऐ", "आई"],
    "similarity order": ["आई", "ऐ", "आई"]
  }},
  "love": {{
    "phonemes": "lˈʌv",
    "choices": ["लव", "लव", "लव"],
    "similarity order": ["लव", "लव", "लव"]
  }},
  "you": {{
    "phonemes": "juː",
    "choices": ["यू", "यू", "यू"],
    "similarity order": ["यू", "यू", "यू"]
  }},
}}
```
"""
for p, ph in enumerate(shfflephonemized):
    start += f"{words[p]}: {ph}\n"
start = start[:-1]
start += f"""
```
Again, the responses should be in a JSON format and sort them in order of the similarity to each phoneme sequence.
{{
"""
for p, ph in enumerate(shfflephonemized):
    # start += f"""  "{ph}": [1st most similar {language} word, 2nd most similar {language} word, 3rd most similar {language} word],\n"""
    # start += f"""  "{ph}": [1st {language} characters, 2nd {language} characters, 3rd {language} characters],\n"""
    start += f"""  "{words[p]}": {{
    "phonemes": "{ph}",
    "choices": [`1st choices of {language} characters`, `2nd choices of {language} characters`, `3rd choices of {language} characters`],
    "similarity order": [`1st most similar {language} characters`, `2nd most similar {language} characters`, `3rd most similar {language} characters`],
    }},\n"""
start = start[:] + "}"
print(start)

In [ ]:
# pip install openai
from openai import OpenAI
api_key = "sk-ANuLBmSVZUkf8I308b17C775Ce464c7dA06dF119E9A29fE3"
api_base = "https://api.ai-gaochao.cn/v1"
client = OpenAI(api_key=api_key, base_url=api_base)
def get_response(text):
    completion = client.chat.completions.create(
      model="gpt-3.5-turbo-1106",
      # model="gpt-3.5-turbo-instruct",
      messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": text}
      ]
    )
    return completion.choices[0].message.content

In [ ]:
# response = get_response(start)

In [ ]:
print(response)

In [ ]:
a_list = [
    eval(response[response.index("{"):-1*response[::-1].index("}")]) if response[-1]!="}" else eval(response[response.index("{"):]),
]
dirs = []
for a in a_list:
    a = {key: a[key] for key in words}
    # a = {key: a[key] for key in shfflephonemized}
    dirs += [a]
for i in range(len(dirs)):
    for key in dirs[i]:
        newlist = []
        for j in range(len(dirs[i][key]["similarity order"])):
            newlist += [dirs[i][key]["similarity order"][j]]*(3-j)
        dirs[i][key]["similarity order"] = newlist
data = {key: [element for i in range(len(dirs)) for element in dirs[i][key]["similarity order"]] for key in dirs[0]}

In [ ]:
rank = 2
arrays = []
counts = []
for i in range(len(data)):
    c = collections.Counter(data[list(data.keys())[i]])
    if len(c)>1:
        df = pd.DataFrame(c.items(), columns=["phonemes", "count"]).sort_values("count", ascending=False).values
        arrays += [list(df[:rank,0])]
        counts += [list(df[:rank,1])]
    else:
        arrays += [list(c.keys())*2]
        counts += [list(c.values())*2]
arrays = np.array(arrays).T
for i in range(rank):
    print(" ".join(list(arrays[i])))

In [ ]:
key = list(data.keys())[0]
c = collections.Counter(data[key])
print(key)
print(" ".join(list(c.keys())))
c

In [ ]:
a_list = [
    {
        "Author": ["오토", "오브", "더"],
        "of": ["오프", "오브", "더"],
        "the": ["디", "더", "덴저"],
        "danger": ["데인저", "덴저", "덴자"],
        "trail": ["트레일", "트렐", "트래"],
        "philip": ["필립", "필립스", "필림"],
        "steels": ["스틸스", "스틸", "스틸즈"],
        "et": ["엣", "에티", "엣또"],
        "cetera": ["세테라", "쎄테라", "쎄터라"]
    },
    {
        "Author": ["오프어", "오브어", "오브더"],
        "of": ["오프", "어프", "어브"],
        "the": ["디", "더", "지"],
        "danger": ["데인저", "대인저", "덴저"],
        "trail": ["트레일", "트레이일", "테일"],
        "philip": ["필립", "피립", "피릅"],
        "steels": ["스틸스", "스틸즈", "스틸쓰"],
        "et": ["엣", "이트", "에트"],
        "cetera": ["세테라", "세테라", "세테라"]
    },
    {
        "Author": ["오파", "어터", "오설"],
        "of": ["오프", "어프", "오브"],
        "the": ["디", "더", "더"],
        "danger": ["데인저", "대인저", "다인저"],
        "trail": ["트레일", "테일", "틀레"],
        "philip": ["필립", "피립", "펠립"],
        "steels": ["스틸스", "스틸즈", "스틸츠"],
        "et": ["엣", "에트", "이트"],
        "cetera": ["세테라", "체테라", "세테라"]
    }
]
dirs = []
for a in a_list:
    a = {key: a[key] for key in words}
    dirs += [a]
for i in range(len(dirs)):
    for key in dirs[i]:
        newlist = []
        for j in range(len(dirs[i][key])):
            newlist += [dirs[i][key][j]]*(3-j)
        dirs[i][key] = newlist
data = {key: [element for i in range(len(dirs)) for element in dirs[i][key]] for key in dirs[0]}

In [ ]:
a_list = [
{
  "Author": ["オーサー", "オーソー", "オウザー"],
  "of": ["オヴ", "オーヴ", "オフ"],
  "the": ["ザ", "ザ・", "ゼ"],
  "danger": ["デンジャー", "ダンジャー", "デンガー"],
  "trail": ["トレイル", "トレール", "トライル"],
  "philip": ["フィリップ", "フィリップ", "フィリプ"],
  "steels": ["スティールズ", "スティールズ", "ステールズ"],
  "et": ["エト", "エット", "イート"],
  "cetera": ["セテラ", "セテラ", "セテラ"]
},
{
  "Author": ["ˈɔːθɚ", "オーサー", "オーソー"],
  "of": ["ʌv", "オブ", "オヴ"],
  "the": ["ðə", "ザ", "ザ"],
  "danger": ["dˈeɪndʒɚ", "デインジャー", "デンジャー"],
  "trail": ["tɹˈeɪl", "トレイル", "トレール"],
  "philip": ["fˈɪlɪp", "フィリップ", "フィリプ"],
  "steels": ["stˈiːlz", "スティールズ", "ステールズ"],
  "et": ["ˈɛt", "エト", "エット"],
  "cetera": ["sˈɛɾɚɹə", "セテラ", "セテラ"]
},
{
  "Author": ["オーサー", "オウサー", "アウサー"],
  "of": ["オブ", "オヴ", "オッブ"],
  "the": ["ザ", "ゼ", "テ"],
  "danger": ["デインジャー", "ダンジャー", "ダンゲル"],
  "trail": ["トレイル", "トライル", "テイル"],
  "philip": ["フィリップ", "フィリプ", "ピリップ"],
  "steels": ["スティールズ", "ステルズ", "スチールズ"],
  "et": ["エト", "エッ", "エ"],
  "cetera": ["セテラ", "セテラー", "セテッラ"]
},
]
dirs = []
for a in a_list:
    a = {key: a[key] for key in words}
    dirs += [a]
for i in range(len(dirs)):
    for key in dirs[i]:
        newlist = []
        for j in range(len(dirs[i][key])):
            ph = dirs[i][key][j]
            if ph==phonemized[words.index(key)]:
                continue
            newlist += [ph]*(3-j)
        dirs[i][key] = newlist
data = {key: [element for i in range(len(dirs)) for element in dirs[i][key]] for key in dirs[0]}

In [ ]:
a_list = [
    {
        "Author": {
            "choices": ["ऑथर", "आथर", "आउथर"],
            "similarity order": ["आथर", "ऑथर", "आउथर"]
        },
        "of": {
            "choices": ["अफ़", "ऑफ़", "आफ़"],
            "similarity order": ["अफ़", "आफ़", "ऑफ़"]
        },
        "the": {
            "choices": ["द", "दा", "ध"],
            "similarity order": ["द", "दा", "ध"]
        },
        "danger": {
            "choices": ["डेंजर", "डैंजर", "डॅन्जर"],
            "similarity order": ["डैंजर", "डेंजर", "डॅन्जर"]
        },
        "trail": {
            "choices": ["ट्रेल", "ट्रेल्स", "त्रैल"],
            "similarity order": ["ट्रेल", "त्रैल", "ट्रेल्स"]
        },
        "philip": {
            "choices": ["फिलिप", "फिलिप्स", "फिलप"],
            "similarity order": ["फिलिप", "फिलप", "फिलिप्स"]
        },
        "steels": {
            "choices": ["स्टील्स", "स्टील", "स्टील्स"],
            "similarity order": ["स्टील्स", "स्टील", "स्टील्स"]
        },
        "et": {
            "choices": ["इट", "इति", "ईट"],
            "similarity order": ["ईट", "इट", "इति"]
        },
        "cetera": {
            "choices": ["सेटेरा", "सेटेरा", "सेतेरा"],
            "similarity order": ["सेटेरा", "सेटेरा", "सेतेरा"]
        }
    },
    {
        "Author": {
            "choices": ["आथर", "ऑथर", "ऑथर्स"],
            "similarity order": ["आथर", "ऑथर", "ऑथर्स"]
        },
        "of": {
            "choices": ["अफ", "ऑफ", "ऑफ्स"],
            "similarity order": ["अफ", "ऑफ", "ऑफ्स"]
        },
        "the": {
            "choices": ["द", "द", "द"],
            "similarity order": ["द", "द", "द"]
        },
        "danger": {
            "choices": ["डैंजर", "डैंजर", "डैंजर्स"],
            "similarity order": ["डैंजर", "डैंजर", "डैंजर्स"]
        },
        "trail": {
            "choices": ["ट्रेल", "ट्रेल", "ट्रेल्स"],
            "similarity order": ["ट्रेल", "ट्रेल", "ट्रेल्स"]
        },
        "philip": {
            "choices": ["फिलिप", "फिलिप", "फिलिप्स"],
            "similarity order": ["फिलिप", "फिलिप", "फिलिप्स"]
        },
        "steels": {
            "choices": ["स्टील्स", "स्टील्स", "स्टील्स"],
            "similarity order": ["स्टील्स", "स्टील्स", "स्टील्स"]
        },
        "et": {
            "choices": ["एट", "ईट", "इट"],
            "similarity order": ["एट", "ईट", "इट"]
        },
        "cetera": {
            "choices": ["सीटेरा", "सीटेरा", "सीटेरा"],
            "similarity order": ["सीटेरा", "सीटेरा", "सीटेरा"]
        }
    },
    {
        "Author": {
            "choices": ["ऑथर", "आथर", "आउथर"],
            "similarity order": ["ऑथर", "आथर", "आउथर"]
        },
        "of": {
            "choices": ["अफ", "ऑफ", "औफ"],
            "similarity order": ["ऑफ", "अफ", "औफ"]
        },
        "the": {
            "choices": ["द", "द", "द"],
            "similarity order": ["द", "द", "द"]
        },
        "danger": {
            "choices": ["डेंजर", "डेन्जर", "डैंजर"],
            "similarity order": ["डेंजर", "डेन्जर", "डैंजर"]
        },
        "trail": {
            "choices": ["ट्रेल", "ट्रेल", "ट्रेल"],
            "similarity order": ["ट्रेल", "ट्रेल", "ट्रेल"]
        },
        "philip": {
            "choices": ["फिलिप", "फिलिप", "फिलिप"],
            "similarity order": ["फिलिप", "फिलिप", "फिलिप"]
        },
        "steels": {
            "choices": ["स्टील्स", "स्टील्स", "स्टील्स"],
            "similarity order": ["स्टील्स", "स्टील्स", "स्टील्स"]
        },
        "et": {
            "choices": ["एट", "ईट", "इट"],
            "similarity order": ["एट", "ईट", "इट"]
        },
        "cetera": {
            "choices": ["सेटेरा", "सेटेरा", "सेटेरा"],
            "similarity order": ["सेटेरा", "सेटेरा", "सेटेरा"]
        }
    },
]
dirs = []
for a in a_list:
    a = {key: a[key] for key in words}
    dirs += [a]
for i in range(len(dirs)):
    for key in dirs[i]:
        newlist = []
        for j in range(len(dirs[i][key]["similarity order"])):
            newlist += [dirs[i][key]["similarity order"][j]]*(3-j)
        dirs[i][key]["similarity order"] = newlist
data = {key: [element for i in range(len(dirs)) for element in dirs[i][key]["similarity order"]] for key in dirs[0]}

In [ ]:
a_list = [
    {
        "Author": {
            "phonemes": "ˈɔːθɚ",
            "choices": ["автор", "офор", "афтар"],
            "similarity order": ["автор", "афтар", "офор"]
        },
        "of": {
            "phonemes": "ʌv",
            "choices": ["ов", "ав", "ув"],
            "similarity order": ["ов", "ав", "ув"]
        },
        "the": {
            "phonemes": "ðə",
            "choices": ["зе", "де", "те"],
            "similarity order": ["де", "зе", "те"]
        },
        "danger": {
            "phonemes": "dˈeɪndʒɚ",
            "choices": ["денджер", "данджер", "дэнджэр"],
            "similarity order": ["денджер", "данджер", "дэнджэр"]
        },
        "trail": {
            "phonemes": "tɹˈeɪl",
            "choices": ["трэйл", "трейл", "траил"],
            "similarity order": ["трейл", "трэйл", "траил"]
        },
        "philip": {
            "phonemes": "fˈɪlɪp",
            "choices": ["филип", "пилип", "филлип"],
            "similarity order": ["филип", "филлип", "пилип"]
        },
        "steels": {
            "phonemes": "stˈiːlz",
            "choices": ["стилз", "стилс", "стелс"],
            "similarity order": ["стилс", "стилз", "стелс"]
        },
        "et": {
            "phonemes": "ˈɛt",
            "choices": ["эт", "ет", "ат"],
            "similarity order": ["эт", "ет", "ат"]
        },
        "cetera": {
            "phonemes": "sˈɛɾɚɹə",
            "choices": ["сетера", "цетера", "ситера"],
            "similarity order": ["сетера", "цетера", "ситера"]
        }
    }

]
dirs = []
for a in a_list:
    a = {key: a[key] for key in words}
    # a = {key: a[key] for key in shfflephonemized}
    dirs += [a]
for i in range(len(dirs)):
    for key in dirs[i]:
        newlist = []
        for j in range(len(dirs[i][key]["similarity order"])):
            newlist += [dirs[i][key]["similarity order"][j]]*(3-j)
        dirs[i][key]["similarity order"] = newlist
data = {key: [element for i in range(len(dirs)) for element in dirs[i][key]["similarity order"]] for key in dirs[0]}